In [2]:
import pandas as pd
import requests
import html5lib
import lxml.html as lh
from bs4 import BeautifulSoup as bs
import time
import datetime
import sys
import pymysql

sys.path.append("C:/Users/etlers/Documents/project/python/common")

import date_util as DU

In [28]:
kospi_url = "https://finance.naver.com/sise/sise_quant_high.nhn?sosok=0"
kosdak_url = "https://finance.naver.com/sise/sise_quant_high.nhn?sosok=1"

# 매수
def buy_n_sell_stock(jongmok_cd):
    
    # 매도
    def sell_stock(sell_price):
        pass
    
    buy_price = 0
    buy_ea = 
    # 시장가로 매수
    # 매수내역을 가져와 매도 금액 지정
    sell_price = 0
    sell_stock(sell_price)


def execute():
    def get_qunat_high(base_url, list_whole):
        response = requests.get( base_url, headers={"User-agent": "Mozilla/5.0"} )
        soup = bs(response.text, 'html.parser')
        list_quant_high = str(soup.findAll("table", {"class": "type_2"})).split('\n')

        start_tf = False
        idx = 0
        list_jongmok = []

        for val_string in list_quant_high:
            strip_val = val_string.strip()
            if '<td class="no">' in strip_val:
                idx = 0
                if (len(list_jongmok) > 0):
                    if (list_jongmok[2].count(' ') == 0 and list_jongmok[4] > 0):
                        list_whole.append(list_jongmok)
                list_jongmok = []
                start_tf = True
            if strip_val[:2] == '</': continue
            if start_tf == False: continue
            if '<td class="number">' == strip_val: continue

            idx += 1

            if idx == 3:
                strip_val = strip_val.split(">")[2].split("<")[0]
            elif idx == 5:
                if 'nv01' in strip_val:
                    num_sign = -1
                else:
                    num_sign = 1
                continue
            elif idx == 6:
                try:
                    strip_val = str(int(strip_val.replace(",","")) * num_sign)
                except:
                    strip_val = "0"
            else:
                try:
                    strip_val = strip_val.split(">")[1].split("<")[0]
                    if len(strip_val) == 0: continue
                except:
                    pass

            try:
                if strip_val.count(".") > 0:
                    strip_val = float(strip_val.replace(",",""))
                else:
                    strip_val = int(strip_val.replace(",",""))
            except:
                strip_val = strip_val.replace("+", "")

            list_jongmok.append(strip_val)

        if (list_jongmok[2].count(' ') == 0 and list_jongmok[4] > 0):
            list_whole.append(list_jongmok)

        return list_whole



    for ii in range(2):
        list_result = []
        # KOSPI Data
        list_result = get_qunat_high(kospi_url, list_result)
        # KOSDAK Data
        list_result = get_qunat_high(kosdak_url, list_result)

        list_cols = ["RANK","증가율","종목명","현재가","전일비","등락률","매수호가","매도호가","거래량","전일거래량","PER"]
        df_sudden = pd.DataFrame(list_result, columns=list_cols)
        df_quant_high = df_sudden.drop(columns=['RANK', "PER"])
        df_quant_high = df_quant_high[
            (df_quant_high["등락률"].str.replace("%","").astype(float) < 7.0) & 
            (df_quant_high["등락률"].str.replace("%","").astype(float) > 3.0) &
            (df_quant_high["매수호가"] > 4999) &
            (df_quant_high["매수호가"] < 10000)
        ]

    #     df_quant_high = df_quant_high[["종목명","증가율","거래량","전일거래량","현재가","매수호가","매도호가","전일비","등락률"]]
        df_quant_high = df_quant_high[["종목명","증가율","현재가","등락률"]]
        df_quant_high["등락률"] = df_quant_high["등락률"].str.replace("%","").astype(float)
        df_quant_high["증가율"] = df_quant_high["증가율"].astype(int)
        df_quant_high = df_quant_high[df_quant_high["증가율"] > 999]
    #     df_quant_high = df_quant_high.sort_values(by=['증가율'], ascending=False)

        nam = ii % 2
        if nam == 0:
            df_pre = df_quant_high
        else:
            df_now = df_quant_high

        # 10초 간격으로 증가한 경우를 추출하기 위함
        time.sleep(1)

    df_joined = pd.merge(
        df_now,
        df_pre,
        how="outer",
        on="종목명",
        suffixes=("_L","_R"),
        indicator=True
    )

    df_joined = df_joined[df_joined["_merge"] == "both"]
    df_joined["증가율"] = df_joined["증가율_R"] - df_joined["증가율_L"]
    df_joined["등락률"] = df_joined["등락률_R"] - df_joined["등락률_L"]
    df_joined["현재가"] = df_joined["현재가_R"] - df_joined["현재가_L"]
    df_joined["현재가율"] = round((df_joined["현재가_R"] - df_joined["현재가_L"]) / df_joined["현재가_L"], 2) * 100
    # 걸러낼 조건을 추가
    # 현재가가 가장 많이 오른 종목을 대상으로 함
    df_target = df_joined.sort_values(by=['현재가율'], ascending=False)

    return df_target

In [32]:
# 5초마다 수행(1분에 4번을 하게 됨)하면서 원하는 종목이 나올 때까지 함
while True:
    now_dt = DU.get_now_datetime_string()
    now_tm = now_dt.split(" ").replace(":","")
    # 10시가 되면 종료
    if now_tm > "100000":
        break
    # 데이터 추출
    df_result = execute()
    # 찾았으면 매수 & 매도 예약
    if len(df_result) > 1:        
        break
    else:
        print(now_dt, "No Target!!")
    # 9시 30분 전에는 1초, 이후 5초
    time.sleep(5)

df_result

,종목명,증가율_L,현재가_L,등락률_L,증가율_R,현재가_R,등락률_R,_merge,증가율,등락률,현재가,현재가율
0,지엠비코리아,1312,7470,5.81,1312,7470,5.81,both,0,0.0,0,0.0
1,유니크,12846,8580,6.45,12846,8580,6.45,both,0,0.0,0,0.0
2,성창오토텍,2696,9710,3.08,2696,9710,3.08,both,0,0.0,0,0.0
3,우수AMS,2604,6040,3.78,2604,6040,3.78,both,0,0.0,0,0.0
4,삼보모터스,2512,7550,6.94,2512,7550,6.94,both,0,0.0,0,0.0
5,제이엔케이히터,1879,7270,3.86,1879,7270,3.86,both,0,0.0,0,0.0
6,알로이스,1198,6430,5.24,1198,6430,5.24,both,0,0.0,0,0.0
7,에스에스알,1167,7260,5.07,1167,7260,5.07,both,0,0.0,0,0.0


In [35]:
DU.get_now_datetime_string()

'2021-06-12 00:14:17'

In [13]:
df_jongmok = pd.read_csv("./csv/jongmok_list.csv", encoding="cp949")
df_jongmok = df_jongmok[["한글 종목약명","단축코드"]]
dict_jongmok = {}
for key, row in df_jongmok.iterrows():
    dict_jongmok[row["한글 종목약명"]] = row["단축코드"]
    
dict_jongmok

{'마이크로컨텍솔': '098120',
 '스카이이앤엠': '131100',
 '포스코엠텍': '009520',
 'AJ네트웍스': '095570',
 'AK홀딩스': '006840',
 'BGF리테일': '282330',
 'BGF': '027410',
 'BNK금융지주': '138930',
 'BYC우': '001465',
 'BYC': '001460',
 'CJ우': '001045',
 'CJ4우(전환)': '00104K',
 'CJ': '001040',
 'CJ씨푸드1우': '011155',
 'CJ씨푸드': '011150',
 'CMG제약': '058820',
 'CS홀딩스': '000590',
 'DB': '012030',
 'DB금융투자': '016610',
 'DB손해보험': '005830',
 'DB하이텍1우': '000995',
 'DB하이텍': '000990',
 'DGB금융지주': '139130',
 'DI동일': '001530',
 'DL우': '000215',
 'DL건설': '001880',
 'DL': '000210',
 'DL이앤씨우': '37550K',
 'DL이앤씨': '375500',
 'DSR': '155660',
 'DSR제강': '069730',
 'E1': '017940',
 'ESR켄달스퀘어리츠': '365550',
 'ES큐브': '050120',
 'F&F': '007700',
 'GS우': '078935',
 'GS건설': '006360',
 'GS': '078930',
 'HDC': '012630',
 'HDC현대산업개발': '294870',
 'IHQ': '003560',
 'JB금융지주': '175330',
 'JW생명과학': '234080',
 'JW중외제약우': '001065',
 'JW중외제약2우B': '001067',
 'JW중외제약': '001060',
 'JW홀딩스': '096760',
 'KB금융': '105560',
 'KB오토시스': '024120',
 'KCTC': '009070',
 '